# Welcome to the Synergy Oneliners
Created by Fredrik Tärnell - 2019-02-20<br>

This notebook contains useful oneliners for particular occations.<br>
**Don't use them without fully understanding them**.<br> Some will wipe resources away no questions asked.

Jupyter Notebooks can be found: https://github.com/frippe75/synergy-jupyter-notebooks<br>
Together with a Docker image with all dependencies inluded and a quickstart guide.


### API information

http://h17007.www1.hpe.com/docs/enterprise/servers/oneview4.1/cic-api/en/index.html <br>



### Python specific
https://hewlettpackard.github.io/python-hpOneView/index.html<br>
https://github.com/HewlettPackard/python-hpOneView<br>

<br>

## Table of content

- **[Networks](#Network)**<br>
   + [List all networks](#Network-show)<br>
   + [Create a single network](#Network-create)<br>
   + [Delete a single network](#Network-delete)<br>
   + [Create multiple networks](#Network-bulk-create)<br>
   + [Delete multiple networks](#Network-bulk-delete)<br>
   + [Delete ALL networks](#Network-delete-all)<br>
- **[Server Profiles](#SP)**<br>
   + [Power On/Off all Profiles](#SP-power)<br>
   + [Delete all Profiles](#SP-delete)<br>
- **[Server Profiles Templates](#SPT)**<br>
   + [Delete all Profiles Templates](#SPT-delete)<br>
- **[Volume Templates](#VolumeTemplates)**<br>
   + [Delete all volume templates](#VolumeTemplates-deleteall)<br>
- **[Volumes](#Volumes)**<br>
   + [Delete all volumes](#Volumes-deleteall)<br>
- **[Appliances](#Appliances)**<br>
   + [eFuse Composer / Image Streamer](#Appliance-efuse)<br>
   + [eFuse Compute module](#Appliance-efuse-compute)<br>
<br><br>
# First section - Simple tasks via Python



### Import the python OneView library with some additional dependencies for this playbook

In [93]:
from hpOneView.oneview_client import OneViewClient
import pandas
from IPython.display import display, Markdown
from pprint import pprint

# Simple helper function to make nicer tables using markdown and the pandas lib
# TODO: get it into helper module that gets imported above 
def my_table(source=dict(), resource="Resource", columns='name'):
    data = pandas.DataFrame.from_dict(source)
    table = data.to_html(index=False,header=True,columns=columns)
    
    display(Markdown('### ' + resource + ' in Synergy Composer at https://' + config['ip']))
    display(Markdown(table))

### Configure your environment

In [83]:
composer_ip = "syn420-dcs.hpedemo.local"
config = {
    "api_version": "800",
    "ip": composer_ip,
    "credentials": {
        "userName": "Administrator",
        "authLoginDomain": "local",
        "password": "HPEc0nverged!"
    }
}

<br><br>
## Login against the Appliance / Composer instance:
create new object "ov" with the config (see above) using the JSON config dict above

In [81]:
ov = OneViewClient(config)

<br><br>
# Networking  <a name="Network"></a>
## Show existing networks  <a name="Network-show"></a>
This example uses Pandas dataframe to create the HTML table<br>
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_html.html?highlight=to_html#pandas.DataFrame.to_html


In [102]:
poc_networks = ov.ethernet_networks.get_all()

#print(poc_networks)

if poc_networks:
    my_table(poc_networks, resource="Subnets", columns=['name','description','vlanId','uri','purpose'])


### Subnets in Synergy Composer at https://syn420-dcs.hpedemo.local

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>name</th>
      <th>description</th>
      <th>vlanId</th>
      <th>uri</th>
      <th>purpose</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>New network</td>
      <td>None</td>
      <td>3000</td>
      <td>/rest/ethernet-networks/42e2a116-053c-4e05-b92...</td>
      <td>General</td>
    </tr>
  </tbody>
</table>

<br><br>
## Create a network  <a name="Network-create"></a>

In [101]:
options = {
    "name": "New network",
    "vlanId": 3000,
    "ethernetNetworkType": "Tagged",
    "purpose": "General",
    "smartLink": False,
    "privateNetwork": False,
    "type" : "ethernet-networkV4",
    "connectionTemplateUri": None
}

ethernet_network = ov.ethernet_networks.create(options)
print("Created ethernet-network " + ethernet_network['name'] + "   URI: " + ethernet_network['uri'] + "\nsuccessfully.")

Created ethernet-network New network   URI: /rest/ethernet-networks/42e2a116-053c-4e05-b924-af57ed0e9a10
successfully.


<br><br>
## Delete a network  <a name="Network-delete"></a>

In [100]:
ov.ethernet_networks.delete(ethernet_network)
print("Delete the network created above: " + ethernet_network['name'])

Delete the network created above: New network


<br><br>
## Create bulk networks <a name="Network-bulk-create"></a>

In [88]:
options_bulk = {
    "vlanIdRange": "30-40,50,100-109,200",
    "purpose": "General",
    "namePrefix": "Bulk-Ethernet",
    "smartLink": False,
    "privateNetwork": False,
    "type": "bulk-ethernet-networkV1",
    "bandwidth": {
        "maximumBandwidth": 10000,
        "typicalBandwidth": 2000
    }
}

# create the networks
ethernet_nets_bulk = ov.ethernet_networks.create_bulk(options_bulk)

# simply display whats exepected
for net in ethernet_nets_bulk: 
  print("created network: " + net['name'] + " URI: " + net['uri'])

created network: Bulk-Ethernet_30 URI: /rest/ethernet-networks/c28a1cb1-3166-4668-9689-ca28d9911881
created network: Bulk-Ethernet_31 URI: /rest/ethernet-networks/4ed3322c-4277-44cb-a6e6-5ecfd08f1629
created network: Bulk-Ethernet_32 URI: /rest/ethernet-networks/5d4140f7-f952-400d-84b0-2c7a3594694c
created network: Bulk-Ethernet_33 URI: /rest/ethernet-networks/6654937f-c3af-4585-8d5c-63a83e30c287
created network: Bulk-Ethernet_34 URI: /rest/ethernet-networks/fe335bab-c756-4f3f-b34b-865e2dbcfb90
created network: Bulk-Ethernet_35 URI: /rest/ethernet-networks/17766169-f230-44d4-a109-5efc186bc614
created network: Bulk-Ethernet_36 URI: /rest/ethernet-networks/d38a6f77-a4dc-441d-9df1-64409803dd38
created network: Bulk-Ethernet_37 URI: /rest/ethernet-networks/644e9b2e-a86b-4631-854c-a029d9a711ea
created network: Bulk-Ethernet_38 URI: /rest/ethernet-networks/b7d3bb25-7d9c-4a5e-8e40-3a29cae8faa5
created network: Bulk-Ethernet_39 URI: /rest/ethernet-networks/cb745e40-4711-4157-8784-e671e8c19747


<br><br>
## Delete the bulks network above  <a name="Network-bulk-delete"></a>

In [89]:
for net in ethernet_nets_bulk:
    ov.ethernet_networks.delete(net)
    print("delete network: " + net['name'])

delete network: Bulk-Ethernet_30
delete network: Bulk-Ethernet_31
delete network: Bulk-Ethernet_32
delete network: Bulk-Ethernet_33
delete network: Bulk-Ethernet_34
delete network: Bulk-Ethernet_35
delete network: Bulk-Ethernet_36
delete network: Bulk-Ethernet_37
delete network: Bulk-Ethernet_38
delete network: Bulk-Ethernet_39
delete network: Bulk-Ethernet_40
delete network: Bulk-Ethernet_50
delete network: Bulk-Ethernet_100
delete network: Bulk-Ethernet_101
delete network: Bulk-Ethernet_102
delete network: Bulk-Ethernet_103
delete network: Bulk-Ethernet_104
delete network: Bulk-Ethernet_105
delete network: Bulk-Ethernet_106
delete network: Bulk-Ethernet_107
delete network: Bulk-Ethernet_108
delete network: Bulk-Ethernet_109
delete network: Bulk-Ethernet_200


<br><br>
## Delete ALL networks in Oneview  <a name="Network-delete-all"></a>
This will **delete ALL networks off all types** so don't run unless you mean it!

In [87]:
all_nets = ov.ethernet_networks.get_all()
for net in all_nets:
    # If the ov command below is commented out it's safe to give a try as a dry-run
    # ov.ethernet_networks.delete(net)
    print("Deleting network: " + net['name'])

Deleting network: VLAN2000_10.48.0
Deleting network: NFS-Simplivity
Deleting network: ESX_vMotion_FrameInternal
Deleting network: VLAN2000_10.48.10
Deleting network: Tunnel01
Deleting network: VLAN400_10.2
Deleting network: ImageStreamer-Boot
Deleting network: VLAN402_10.3


<br><br>
## Server Profile Templates <a name="SPT"></a>

### Delete ALL existing Server Profile Templates <a name="SPT-delete"></a>

WARNING: Will delete ALL profiles templates unconditionally!

In [15]:
# get a list of all the spts
spts = ov.server_profile_templates.get_all()

# delete them one-by-one
for spt in spts: 
  print("Deleting the Server Profile Template: " + spt['name'] )
  ov.server_profile_templates.delete(spt)


Deleting the Server Profile Template: ESX-1033
Deleting the Server Profile Template: ESX-ny-padwick
Deleting the Server Profile Template: ESXi-SY480-gen10-manual
Deleting the Server Profile Template: ESX-1225-noVsets
Deleting the Server Profile Template: Golden


<br><br>
## Server Profiles <a name="SP"></a>

### Delete ALL existing Server Profiles <a name="SP-delete"></a>

WARNING: Will delete ALL profiles unconditionally!

In [106]:
# get a list of all the sp's
sps = ov.server_profiles.get_all()

# delete them one-by-one
for sp in sps: 
  print("Deleting the Server Profile: " + sp['name'] )
  ov.server_profiles.delete(sp)


Deleting the Server Profile: golden


HPOneViewTaskError: Delete OS volume failed.

### Power on/off  ALL existing Server Profiles <a name="SP-power"></a>

**Warning:** Will power on or off all profiles depending on the new_state variable

In [49]:
# what powerstate to achieve
new_state = "Off" # it's either "On" or "Off"

# get a list of all the sp's
sps = ov.server_profiles.get_all()

# same config for all hardware
configuration = {
        "powerState": new_state,
        "powerControl": "MomentaryPress"
    }

# power them OFF them one-by-one
for sp in sps: 
  # print them nicely
  # pprint(sp)
  sp_uri = sp['serverHardwareUri']
  sp_name = sp['name']
  # remember some profiles be unassigned. i.e they dont have a hardware uri
  if sp_uri is not None:
      print("Powering OFF the Server Profile: " + sp_name + " using ServerHardwareUri " +  sp_uri ) 
      try:
          ov.server_hardware.update_power_state(configuration,  sp_uri)
          print("Successfully changed the power state of '" + sp_name + "' to " + configuration['powerState'])
      except HPOneViewException as e:
          print(e.msg)
  else:
      print("Server Profile: " + sp_name + " is unassigned")
      

Powering OFF the Server Profile: 4 using ServerHardwareUri /rest/server-hardware/39313738-3534-5A43-4A38-313730543343
Successfully changed the power state of '4' to Off
Server Profile: 1 is unassigned
Powering OFF the Server Profile: Golden using ServerHardwareUri /rest/server-hardware/39313738-3534-5A43-4A38-343330373357
Successfully changed the power state of 'Golden' to Off
Powering OFF the Server Profile: 5 using ServerHardwareUri /rest/server-hardware/36343537-3338-4E43-3736-303230383036
Successfully changed the power state of '5' to Off
Powering OFF the Server Profile: 3 using ServerHardwareUri /rest/server-hardware/39313738-3534-5A43-4A38-313730543342
Successfully changed the power state of '3' to Off
Powering OFF the Server Profile: 6 using ServerHardwareUri /rest/server-hardware/36343537-3338-4E43-3736-303230355A46
Successfully changed the power state of '6' to Off
Powering OFF the Server Profile: 2 using ServerHardwareUri /rest/server-hardware/39313738-3534-5A43-4A38-34333037

<br><br>
## Volumes <a name="Volumes"></a>

### Delete all Volumes <a name="Volume-deleteall"></a><br>
**WARNING**: This really deletes all volumes, actual command commented out.

In [20]:
# get a list of all the storage templates
volumes = ov.volumes.get_all()
# pprint(volumes)

# delete them one-by-one
for volume in volumes: 
  print("Deleting the Volume: " + volume['name'] )
  # ov.volumes.delete(volume)

<br><br>
## Volume Templates <a name="VolumeTemplate"></a>

### Delete all Volume Templates <a name="VolumeTemplate-deleteall"></a><br>

In [21]:
# get a list of all the storage templates
vol_temps = ov.storage_volume_templates.get_all()

# delete them one-by-one
for vol_temp in vol_temps: 
  print("Deleting the Volume Template: " + vol_temp['name'] )
  # ov.storage_volume_templates.delete(vol_temp)

Deleting the Volume Template: Volume root template for StoreServ 3.1.3


HPOneViewException: ("The template with id 'e8af6795-e9fa-4a75-805f-aa0500a48d47' is a root template which cannot be removed.", {'errorSource': None, 'nestedErrors': [], 'errorCode': 'STRM_BAD_REQUEST', 'data': {}, 'recommendedActions': ['If you are attempting to disallow provisioning from root templates, set the "StorageVolumeTemplateRequired" global setting to "true".'], 'details': None, 'message': "The template with id 'e8af6795-e9fa-4a75-805f-aa0500a48d47' is a root template which cannot be removed."})

<br><br>
## Logical Interconnect Groups <a name="LIG"></a>

### Add a network to an existing LIGs uplinkset <a name="LIG-uplink-network"></a><br>

In [76]:
lig_name = "VC-LIG01"
lig_uplink_name = "Test"
eth_name = "VLAN400_10.2"

# Get the network
eth = ov.ethernet_networks.get_by('name', eth_name)[0]

# Get Logical Interconnect Group by property
lig = ov.logical_interconnect_groups.get_by('name', lig_name)[0]

lig_uplinksets = lig['uplinkSets']
for uplinkset in lig_uplinksets:
    #pprint(uplinkset)
    if uplinkset['name'] == lig_uplink_name:
        uplinkset['networkUris'].append(eth['uri'])
        ov.logical_interconnect_groups.update(lig)
    else:
        print("Could not find the uplinkset (by_name)")     

"""   
# Get an uplink set resource by name
print("\nGet uplink set by name")
uplink_set = ov.uplink_sets.get_by('name', 'Test')
pprint(uplink_set)

# Add an ethernet network to the uplink set
# To run this example you must define an ethernet network uri or ID below
ethernet_network_id = None
if ethernet_network_id:
    print("\nAdd an ethernet network to the uplink set")
    uplink_set = ov.uplink_sets.add_ethernet_networks(created_uplink_set['uri'], ethernet_network_id)
    print("The uplink set with name = '{name}' have now the networkUris:\n {networkUris}".format(**uplink_set))

"""

'   \n# Get an uplink set resource by name\nprint("\nGet uplink set by name")\nuplink_set = ov.uplink_sets.get_by(\'name\', \'Test\')\npprint(uplink_set)\n\n# Add an ethernet network to the uplink set\n# To run this example you must define an ethernet network uri or ID below\nethernet_network_id = None\nif ethernet_network_id:\n    print("\nAdd an ethernet network to the uplink set")\n    uplink_set = ov.uplink_sets.add_ethernet_networks(created_uplink_set[\'uri\'], ethernet_network_id)\n    print("The uplink set with name = \'{name}\' have now the networkUris:\n {networkUris}".format(**uplink_set))\n\n'

<br><br>
## Appliances <a name="Appliances"></a>

### eFuse an Appliance in bay 1/2 example <a name="Appliance-efuse"></a><br>
Table from 4.1 API spec on how to target different resources within the enclosure.<br>
http://h17007.www1.hpe.com/docs/enterprise/servers/oneview4.1/cic-api/en/index.html#rest/enclosures
<img src="images/EnclosurePatchTable.PNG" height="600" width="800" align="left">

In [105]:
# WARNINIG: Make sure you target the correct enclosure as well as the operation you want! Pay attention to the table above
#           The actual patch request is commented out below for safety.
#           This is not something you normally do, only on request from support

enc_name = "HPEDemo-POCframe01"

# Get the Enclosure
efuse_enc = ov.enclosures.get_by('name', enc_name)

# If needed pretty print the entire JSON 
#pprint (efuse_enc)

# Grab the first enclosure uri
enc_uri = efuse_enc[0]['uri']

# Setup your patch request
patch_op = "replace"
#patch_path = "/applianceBays/1/bayPowerState"  # In a single POC frame this would be the Composer i.e Bay1
patch_path = "/applianceBays/2/bayPowerState"   # In a single POC frame this would be the Image Streamer i.e Bay2
patch_value = "E-Fuse" # Not the same for all paths

print("Request a PATCH on resource " + enc_uri + " using " + "op=" + patch_op + " path=" + patch_path + " val=" + patch_value)
#patched_end = ov.enclosures.patch(enc_uri, patch_op, patch_path, patch_value)


Request a PATCH on resource /rest/enclosures/000000CN754602DT using op=replace path=/applianceBays/2/bayPowerState val=E-Fuse


{'type': 'EnclosureV7',
 'uri': '/rest/enclosures/000000CN754602DT',
 'category': 'enclosures',
 'eTag': '2019-03-01T15:47:47.138Z',
 'created': '2019-02-20T14:21:26.498Z',
 'modified': '2019-03-01T15:47:47.138Z',
 'refreshState': 'NotRefreshing',
 'stateReason': 'Configured',
 'enclosureType': 'SY12000',
 'enclosureTypeUri': '/rest/enclosure-types/SY12000',
 'enclosureModel': 'Synergy 12000 Frame',
 'uuid': '000000CN754602DT',
 'serialNumber': 'CN754602DT',
 'partNumber': '000000-010',
 'reconfigurationState': 'NotReapplyingConfiguration',
 'uidState': 'Off',
 'licensingIntent': 'NotApplicable',
 'deviceBayCount': 12,
 'deviceBays': [{'type': 'DeviceBayV400',
   'bayNumber': 1,
   'model': None,
   'devicePresence': 'Present',
   'profileUri': None,
   'deviceUri': '/rest/drive-enclosures/CN7601029D',
   'coveredByProfile': None,
   'coveredByDevice': '/rest/drive-enclosures/CN7601029D',
   'ipv4Setting': None,
   'uri': None,
   'category': 'device-bays',
   'eTag': None,
   'created

<br><br>
### eFuse a compute module in bay6 in a particular frame <a name="Appliance-efuse-compute"></a><br>

In [108]:
# WARNINIG: Make sure you target the correct enclosure as well as the operation you want! Pay attention to the table above
#           The actual patch is commented out below for safety.
#           This is not something you normally do, only on request from support

enc_name = "HPEDemo-POCframe01"

# Get the Enclosure
efuse_enc = ov.enclosures.get_by('name', enc_name)

# If needed pretty print the entire JSON 
#pprint (efuse_enc)

# Grab the first enclosure uri
enc_uri = efuse_enc[0]['uri']

# Setup your patch request
patch_op = "replace"
patch_path = "/deviceBays/6/bayPowerState"   # In a single POC frame this would be the Computemodule in Bay6
patch_value = "E-Fuse" # Not the same for all paths

print("Request a PATCH on resource " + enc_uri + " using " + "op=" + patch_op + " path=" + patch_path + " val=" + patch_value)
#patched_end = ov.enclosures.patch(enc_uri, patch_op, patch_path, patch_value)

Request a PATCH on resource /rest/enclosures/000000CN754602DT using op=replace path=/deviceBays/6/bayPowerState val=E-Fuse
